In [1]:
import sys,os,re,string
import csv
from itertools import combinations, chain, islice
from collections import Counter

import urllib
import numpy as np
import time
import pandas as pd

import json
import datetime
import pylab as pl

from pprint import pprint
from bs4 import BeautifulSoup as bs
import inspect



cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]) + 
                              "/word_cloud-master")
if cmd_folder not in sys.path:
     sys.path.insert(0, cmd_folder)


sys.path.append(os.getcwd() + "/word_cloud-master")
print (sys.path)
from wordcloud import make_wordcloud
from sklearn.feature_extraction.text import CountVectorizer
from config import tvsfile
from argparse import ArgumentParser


['/Users/fbianco/science/Dropbox/LSST_TVS/TVSaffiliations/word_cloud-master', '', '//anaconda/envs/p2nb/lib/python2.7/site-packages/uncertainties-2.4.6.1-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/rpy2-2.8.0.dev0-py2.7-macosx-10.5-x86_64.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/MiniSom-0.1-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/sklearn-0.0-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/pyOpenSSL-16.0.0-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/cryptography-1.3.1-py2.7-macosx-10.5-x86_64.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/cffi-1.5.2-py2.7-macosx-10.5-x86_64.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/ipaddress-1.0.16-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/enum34-1.1.2-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/idna-2.1-py2.7.egg', '//anaconda/envs/p2nb/lib/python2.7/site-packages/pycparser-2.14-py2.7.egg', '//anaconda/envs

In [23]:
def most_common(lst):
    return max(set(lst), key=lst.count)

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
class OrderedTuple(tuple):

    def __init__(self, init=()):
        if init:
            init = list(init)
            init.sort()
            init = tuple(init)
            self._oinit = init
            tuple.__init__(init)
        else:
            tuple.__init__(self)

    def __eq__(self, other):
        return self._oinit[0] == other._oinit[0] and self._oinit[1] == other._oinit[1]
        # return self._oinit == other._oinit

    def __hash__(self):
        return hash(self._oinit)

    def __repr__(self):
        return repr(self._oinit)

def mycounter(s):
    mydic = {}
    for w in s.split():
        #print w
#        print len(re.findall(w, s))
        if len(w)<3:
            continue
        if w in mydic.keys():
            mydic[w]=mydic[w]+1
        else:
            mydic[w]=1
#        try:
#            mydic[w] = len(re.findall(w,s))
#        except:
#            print  len(re.findall(w,s))
#            pass

    #print mydic
    return mydic

def parse_args(subglist):
    """ Use ArgParser to build up the arguments we will use in our script
    """
    stored_args = {}
    # get the script name without the extension & use it to build up
    # the json filename
    parser = ArgumentParser(description='Selecting members by subgroup')
    parser.add_argument('subgroup',
                        action='store',
                        default=None,
                        help='Choose the subgroup affiliation:' +
                        ' -- '.join([s for s in subglist]))

    args = parser.parse_args()

    return args


In [36]:
pl.ion()
print (tvsfile)
if tvsfile is None:
    print ("Required Argument: Google Doc file identifier (if you do not have it email federica!)")
    sys.exit()
    

TVSMembers = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                        tvsfile +
                        '/export?gid=0&format=csv',
                        index_col=0)
TVSMembers.head()
TVSMembers.tail()

1Q8YPGxh-Ebs02ykfGypj7oThQrgqKzrU6qyIuPT3bhY


,first name,affiliation,email,primary,secondary,secondary.1,secondary.2,Unnamed: 8,Unnamed: 9
last name,,,,,,,,,
Rodriguez,Joseph,Vanderbilt,rodriguez.jr.joey@gmail.com,Transiting Planets,NaN,NaN,NaN,NaN,NaN
Sasselov,Dimitar,Harvard Smithsonian Center for Astrophysics,sasselov@cfa.harvard.edu,Transiting Planets,Distance Scales,Pulsating Variables,NaN,NaN,NaN
Shporer,Avi,Caltech,shporer@gps.caltech.edu,Transiting Planets,Interacting Binaries,NaN,NaN,NaN,NaN
Stassun,Keivan,Vanderbilt,keivan.stassun@vanderbilt.edu,Transiting Planets,Galactic,Classification/Characterization,Magnetically Active Stars,NaN,NaN
Tejos,Nicolas,NaN,ntejos@ucolick.org,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
tvsgroups = TVSMembers.primary.unique()
subgroups = TVSMembers.primary
subgroups  = pd.concat([TVSMembers.primary, TVSMembers.secondary, TVSMembers['secondary.1'], TVSMembers['secondary.2']])

In [26]:
subgroups.size

632

In [27]:
#content = [[TVSMembers.primary.values[i]] for i in range(TVSMembers.size)]
len(TVSMembers.primary.values), len(TVSMembers.index)

(158, 158)

In [37]:
 
mywords=' '.join([sg.replace(' ', '').replace('/', '') for sg in subgroups if not isinstance(sg, float)])#[inner for outer in content for inner in outer ])
content = [[TVSMembers.primary.values[i], TVSMembers.secondary.values[i], 
            TVSMembers['secondary.1'].values[i], 
            TVSMembers['secondary.2'].values[i]] for i in range(len(TVSMembers.index))]
mycounts=mycounter(mywords)        
words=[]
counts=[]
for k in mycounts.iterkeys():
    words.append(k)
    counts.append(mycounts[k])
    
f = open('data/affiliationsNumbers.csv','w')
f.write("subg,tot\n")
for k in mycounts:
    f.write(k + ",%d\n"%mycounts[k])
f.write("totalaff" + ",%d\n"%np.array(mycounts.values()).sum())
f.write("totalmemb" + ",%d\n"%len(TVSMembers.index))
f.close()

In [29]:
words=np.array(words)
counts=np.array(counts)
# throw away some words, normalize
words = words[counts > 1]
counts = counts[counts > 1]
output_filename = "%s/cloud.png"%datetime.date.today()
os.system("mkdir %s"%datetime.date.today())
counts = make_wordcloud(words, counts, output_filename)

('DroidSansMono.ttf', 461)
('DroidSansMono.ttf', 460)
('DroidSansMono.ttf', 459)
('DroidSansMono.ttf', 458)
('DroidSansMono.ttf', 457)
('DroidSansMono.ttf', 456)
('DroidSansMono.ttf', 455)
('DroidSansMono.ttf', 454)
('DroidSansMono.ttf', 453)
('DroidSansMono.ttf', 452)
('DroidSansMono.ttf', 451)
('DroidSansMono.ttf', 450)
('DroidSansMono.ttf', 449)
('DroidSansMono.ttf', 448)
('DroidSansMono.ttf', 447)
('DroidSansMono.ttf', 446)
('DroidSansMono.ttf', 445)
('DroidSansMono.ttf', 444)
('DroidSansMono.ttf', 443)
('DroidSansMono.ttf', 442)
('DroidSansMono.ttf', 441)
('DroidSansMono.ttf', 440)
('DroidSansMono.ttf', 439)
('DroidSansMono.ttf', 438)
('DroidSansMono.ttf', 437)
('DroidSansMono.ttf', 436)
('DroidSansMono.ttf', 435)
('DroidSansMono.ttf', 434)
('DroidSansMono.ttf', 433)
('DroidSansMono.ttf', 432)
('DroidSansMono.ttf', 431)
('DroidSansMono.ttf', 430)
('DroidSansMono.ttf', 429)
('DroidSansMono.ttf', 428)
('DroidSansMono.ttf', 427)
('DroidSansMono.ttf', 426)
('DroidSansMono.ttf', 425)
(

In [30]:
_t = time.time()
wordlists={}
i=0
#print content
print (content)
for row in content:
    i+=1
    wordlists[i] = {}
    #print ([s for s in row if s != '' and not isinstance(s, float)])
    try:      
        thisset = set([s for s in row if s != '' and not isinstance(s, float)])
    except TypeError: 
        continue

    wordlists[i]['words'] = thisset
    _tuples = combinations(wordlists[i]['words'], 2)
    tuples = set()
    for t in _tuples:
        tuples.add(OrderedTuple(t))
    wordlists[i]['tuples'] = tuples # unique ordered tuples
    wordlists[i]['hashes'] = set([ hash(t) for t in tuples ])
    assert len(tuples) == len(wordlists[i]['hashes'])
    # print('%s, %s' % (user, len(tuples)))
delta = time.time()-_t
print('\n%d data collected in %dsec.' % (i, delta))

[['Blazars', 'Multiwavelength Characterization/Counterparts', 'Tidal Disruption Events', 'Interacting Binaries'], ['Classification/Characterization', 'Fast Transients', 'Tidal Disruption Events', nan], ['Classification/Characterization', 'Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Distance Scales'], ['Classification/Characterization', nan, nan, nan], ['Classification/Characterization', 'Supernovae', 'Interacting Binaries', 'Pulsating Variables'], ['Classification/Characterization', 'Galactic', nan, nan], ['Classification/Characterization', nan, nan, nan], ['Classification/Characterization', 'Multiwavelength Characterization/Counterparts', 'Blazars', 'Gravitational Waves'], ['Classification/Characterization', 'Fast Transients', 'Galactic', 'Gravitational Waves'], ['Classification/Characterization', 'Cosmological', 'Distance Scales', 'Supernovae'], ['Classification/Characterization', 'Fast Transients', 'Galactic', 'Pulsating Variables'], ['Classification/Characte

In [22]:
wordlists

{1: {'hashes': {-7271856045397031115,
   -6475606560907208963,
   -6303685548467894934,
   -5398552806674909790,
   7148511812509456546,
   8809723922306783285},
  'tuples': {('Blazars', 'Tidal Disruption Events'),
   ('Blazars', 'Interacting Binaries'),
   ('Interacting Binaries', 'Tidal Disruption Events'),
   ('Blazars', 'Multiwavelength Characterization/Counterparts'),
   ('Interacting Binaries', 'Multiwavelength Characterization/Counterparts'),
   ('Multiwavelength Characterization/Counterparts', 'Tidal Disruption Events')},
  'words': {'Blazars',
   'Interacting Binaries',
   'Multiwavelength Characterization/Counterparts',
   'Tidal Disruption Events'}},
 2: {'hashes': {-7460472462923700992,
   4055671811984229135,
   8895283935420967708},
  'tuples': {('Classification/Characterization', 'Fast Transients'),
   ('Fast Transients', 'Tidal Disruption Events'),
   ('Classification/Characterization', 'Tidal Disruption Events')},
  'words': {'Classification/Characterization',
   'Fast

In [ ]:
import itertools
tmp = list(itertools.combinations(content[0], 2))
for row in content[1:]:
    tmp = tmp + list(itertools.combinations(row, 2))
print tmp

In [ ]:
alllist = set(sum([[stat[0][0],stat[0][1]] for stat in stats if  stat[1]>=2],[]))
allpairs = list(itertools.combinations(list(alllist), 2))

In [ ]:
iterators = []
for user in wordlists.iterkeys():
    iterators.append(iter(wordlists[user]['tuples']))
#print (iterators)
hash_dict = {}
# { hash_value: OrderTuple, ... }
print('Making hash_dict ...')
_t=time.time()
i=0
for _tuple in chain(*iterators):
    i+=1
    hash_dict[hash(_tuple)] = _tuple
    if (i%100000 == 0):
        sys.stdout.write('\r%d' % i)
        sys.stdout.flush()
print('\n%d hash_dict done in %dsec.' % (i, delta))
    

iterators = []
for user in wordlists.iterkeys():
    iterators.append(iter(wordlists[user]['hashes']))
_stats = {}
done = []
i=0

print('Counting ...')
_t = time.time()
for t in chain(*iterators):
    if t in done:
        continue
    i+=1
    if (i%1000 == 0):
        sys.stdout.write("\r%s" % i)
        sys.stdout.flush()
    done.append(t)
    _count = 0
    for user in wordlists.iterkeys():
        if t in wordlists[user]['hashes']:
            _count += 1
    _stats[hash(t)]=_count

delta = time.time() - _t
print('\nDone writing %d in %dsec.' % (i, delta))

print('Preparing data to write ... ')
stats = []
error_hashes = []
for _hash, _tuple in hash_dict.iteritems():
    try:
        _count = _stats[_hash]
    except KeyError as e:
        error_hashes.append(e)
    else:
        stats.append((_tuple, _count))
print('error_hashes=%r' % len(error_hashes))

In [ ]:
allstats = {}

for stat in stats:
    print (stat, allstats.keys())
    if 1: #not stat[0][0] in allstats.keys():
        allstats[stat[0][0]] = {}
        allstats[stat[0][0]][stat[0][1]] = stat[1]
        #if 'star' in stat[0][0]:
        #    print allstats[stat[0][0]]
#       for i in range(stat[1]):

#    raw_input()
print (allstats)

In [ ]:
alllist = set(sum([[stat[0][0],stat[0][1]] for stat in stats if  stat[1]>=2],[]))
print alllist,len(alllist)
mymatrix=np.zeros((len(alllist),len(alllist)),int)
print mymatrix.shape
import csv


In [ ]:
for f in allpairs: 
    print tmp.count(f), tmp.count(f[::-1])

In [ ]:
with open('data/affiliations.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ', quotechar=' ',quoting=csv.QUOTE_NONE,escapechar='\\')
        spamwriter.writerow(['word1,word2,count'])
        for i,word1 in enumerate(alllist):
            for j,word2 in  enumerate(alllist):
                if word1==word2:
                    mymatrix[i][j]=0
                    spamwriter.writerow( ['"%s","%s",%d,%d'%(word1.replace(" ",""),word2.replace(" ",""),0,n)])
#                    spamwriter.writerow( ['"%s","%s",%d,%d'%(word1,word2,0,subjects[word1])])
                else:
                    
                    cnt = tmp.count((word1,word2)) +  tmp.count((word2,word1))
                    #print ('%s,%s,%s' %(word1, word2, cnt))
                    try: 

In [ ]:
with open('data/affiliations.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ', quotechar=' ',quoting=csv.QUOTE_NONE,escapechar='\\')
        spamwriter.writerow(['word1,word2,count'])
        for i,word1 in enumerate(alllist):
            for j,word2 in  enumerate(alllist):
                if word1==word2:
                    mymatrix[i][j]=0
                    spamwriter.writerow( ['"%s","%s",%d'%(word1.replace(" ",""),word2.replace(" ",""),0)])
#                    spamwriter.writerow( ['"%s","%s",%d,%d'%(word1,word2,0,subjects[word1])])
                else:
                    
                    cnt = tmp.count((word1,word2)) +  tmp.count((word2,word1))
                    #print ('%s,%s,%s' %(word1, word2, cnt))
                    try: 
                        print (cnt)
                    except: 
                        print ("not there")
                        pass
                    try:  
                        mymatrix[i][j] = cnt
                        spamwriter.writerow( ['"%s","%s",%d'%(word1.replace(" ",""), word2.replace(" ",""), cnt)])
                    except: 
                        mymatrix[i][j]=0
                        spamwriter.writerow( ['"%s","%s",%d,'%(word1.replace(" ",""), word2.replace(" ",""), 0)])


#        for stat in stats:
#            if stat[0][0] not in  commonastrowords and not is_number(stat[0][0]) and stat[0][1] not in   
#commonastrowords and not is_number(stat[0][1]) and len(stat[0][0])>=3 and len(stat[0][1])>=3  and stat[1]>1:


#    for word1 in alllist:
#        print word1,
#    for  i,word1 in enumerate(alllist):
#        print word1,"\t\t[" ,
#        for j,word2 in enumerate(alllist):
#            print mymatrix[i][j],",",
#        print "],"

#    print('Writing data')
#    with open('stats.json', 'w') as fo:
#        fo.write(json.dumps(stats))
print('DONE.')